In [18]:
import ipy_table
from collections import Counter
%run ../aux/postgres.py

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import sklearn.cross_validation
from sklearn.cross_validation import train_test_split
from IPython.core.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

#%run functions/preproc.py
#%run functions/classificacao.py

postgres functions loaded...


In [19]:
def stemming_tokenizer(text):
    stemmer = PorterStemmer()
    return [stemmer.stem(w) for w in word_tokenize(text)]




In [20]:
from nltk.stem.snowball import PortugueseStemmer
stemmer = PortugueseStemmer()
stemmer.stem("dogs")

print(stemmer.stem('gatos'))
print(stemmer.stem('gata'))
print(stemmer.stem('gato'))

gat
gat
gat


In [21]:
cmd = ("SELECT t.text, "
       "CASE "
       "WHEN cl.codclassificacao = 230 THEN 226 "
       "ELSE cl.codclassificacao END, "
       "CASE "
       "WHEN cl.codclassificacao = 230 THEN 'service' "
       "ELSE cl.descricao END "
       "FROM analisetweet at "
       "INNER JOIN classificacao cl ON cl.codclassificacao = at.codclassificacao "
       "INNER JOIN tweet t on at.codtweet = t.codtweet "
       "WHERE codanaliseamostra = 126 AND at.codclassificacao <> 36 order by random()")

res01 = query(cmd)
topics = [x[2] for x in res01]
count01 = Counter(topics)

tot = 0
for x in count01.most_common():
    print(x)
    tot = tot + x[1]
print(tot)

tweets = [x[0] for x in res01]
print(len(tweets))

('service', 420)
('clinical', 172)
('personal', 138)
('product', 107)
('information', 94)
('sale', 66)
997
997


In [22]:
topicsID = [x[1] for x in res01]
tweets = [x[0] for x in res01]

print(topicsID[:10])

print(tweets[:10])
print(len(topicsID))
print(len(tweets))


[226, 229, 223, 226, 228, 226, 224, 226, 227, 223]
["RT @CharlesHDean: A great resource if you're curious about dental implants. https://t.co/WRHJKNHGfR", 'Melbourne dental implant failure and how to avoid it https://t.co/620oDESzyg https://t.co/3qKZO6EVVn', 'RT @GuyKawasaki: Ceramic dental implants get crushed by a 20-ton press https://t.co/YVyROKk9bu https://t.co/m1pK8BJM4o', 'RT @ukbestdentist: Full Set Dental Implants in The Blythe #Complete #Teeth #Implants #The #Blythe https://t.co/t7Qc2PiRjI', 'AZDENT Dental Implant Motor System LED Surgical Brushless Handpiece https://t.co/rgsDt2UF8a https://t.co/84NSsVvcYF', '#Dental Implants at Crown House https://t.co/0SFiJXtq0S #dentist #Egham #Surrey https://t.co/nWnt9wE9aT', 'The major difference between traditional dental implants and mini implants is the fact that mini implants are about… https://t.co/zcQKFP6V7h', 'RT @MrG_Picks: https://t.co/UzXkow2y8u Dental implants are a small titanium post that replaces the root of a missing tooth 

## Pré Proc

In [23]:
import re
happyFace = ['😅', '😁', '😍', '❤', '😂', '😋', '😌', '☺', '😘']
sadFace = ['😪', '😭', '😩']
neutralFace = ['😐', '😒']
scaryFace = ['😱']

def proc_emoji(tweet):
    for em in happyFace:
        tweet = tweet.replace(em, ' happyFace ')

    for em in neutralFace:
        tweet = tweet.replace(em, ' neutralFace ')

    for em in sadFace:
        tweet = tweet.replace(em, ' sadFace ')

    for em in scaryFace:
        tweet = tweet.replace(em, ' scaryFace ')
    return tweet
    
    
def pre_proc(doc_complete):
    doc_complete = [ t.lower() for t in doc_complete ]

    # Interrogações
    doc_complete = [re.sub("[$]", " moneymark ", x) for x in doc_complete ]

    # mention removal
    doc_complete = [re.sub("@\\w+", " mentionname ", x) for x in doc_complete ]
    # Remove URL
    doc_complete = [re.sub("(f|ht)(tp)(s?)(://)(.*)[.|/](.*)", " urlurl ", x) for x in doc_complete ]
    # Remove HTML symbols
    doc_complete = [re.sub("&[^\\s]*;", " ", x) for x in doc_complete ]
    # Remove numbers
    doc_complete = [re.sub("[0-9]+", " numbers ", x) for x in doc_complete ]
    # Interrogações
    doc_complete = [re.sub("[?]", " questionmark ", x) for x in doc_complete ]
    # Remove Punctuation    
    doc_complete = [re.sub("[.,\\/#!?+$–\"|%-\\^&\\*;:{}=\\-_`~()]", " ", x) for x in doc_complete ]
    # Remove Emojis
    doc_complete = [proc_emoji(x) for x in doc_complete ]
    
    pattern = re.compile('[\W_]+')
    doc_complete = [pattern.sub(' ', x) for x in doc_complete ]
    return doc_complete

In [24]:
tweetsPP = pre_proc(tweets)
for t, p in zip(tweets[:1000], tweetsPP[:1000]):
    print("tw: [%s]" % t)
    print("pp: [%s]" % p)
    print()

tw: [RT @CharlesHDean: A great resource if you're curious about dental implants. https://t.co/WRHJKNHGfR]
pp: [rt mentionname a great resource if you re curious about dental implants urlurl ]

tw: [Melbourne dental implant failure and how to avoid it https://t.co/620oDESzyg https://t.co/3qKZO6EVVn]
pp: [melbourne dental implant failure and how to avoid it urlurl ]

tw: [RT @GuyKawasaki: Ceramic dental implants get crushed by a 20-ton press https://t.co/YVyROKk9bu https://t.co/m1pK8BJM4o]
pp: [rt mentionname ceramic dental implants get crushed by a numbers ton press urlurl ]

tw: [RT @ukbestdentist: Full Set Dental Implants in The Blythe #Complete #Teeth #Implants #The #Blythe https://t.co/t7Qc2PiRjI]
pp: [rt mentionname full set dental implants in the blythe complete teeth implants the blythe urlurl ]

tw: [AZDENT Dental Implant Motor System LED Surgical Brushless Handpiece https://t.co/rgsDt2UF8a https://t.co/84NSsVvcYF]
pp: [azdent dental implant motor system led surgical brushless h

## Pré Proc 2

In [25]:
from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()
stemmer.stem("dogs")

'dog'

In [26]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
from collections import Counter

stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
#doc_complete = [ t[0] for t in tweets ]
#print(stop)

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop and len(i) > 3])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(stemmer.stem(word) for word in punc_free.split())
    #normalized = [ w for w in punc_free if len(w) > 3 ]
    #normalized = punc_free
    return normalized
''

doc_clean = [clean(doc).split() for doc in tweetsPP] 
#print(doc_clean[:20])


print(len(doc_clean)*0.4)
ws = []
for f in doc_clean:
    for w in f:
        ws.append(w)
cws = Counter(ws)
rWords = [ x[0] for x in cws.most_common() ]
for x in cws.most_common(30):
    print(x)


398.8
('urlurl', 847)
('number', 320)
('mentionnam', 258)
('questionmark', 162)
('teeth', 122)
('smile', 80)
('miss', 71)
('dentist', 69)
('tooth', 56)
('cost', 50)
('restor', 45)
('replac', 44)
('dentistri', 44)
('market', 40)
('need', 35)
('best', 34)
('know', 34)
('patient', 32)
('surgeri', 32)
('learn', 30)
('look', 30)
('help', 30)
('dentur', 29)
('blog', 28)
('cosmet', 27)
('offer', 27)
('call', 26)
('treatment', 26)
('find', 26)
('read', 25)


In [27]:
tweetsClean = []
for t in doc_clean:
    tweetsClean.append(" ".join([i for i in t]))

In [28]:
tweetsPP = pre_proc(tweets)
for t, p, dc, tc in zip(tweets[:10], tweetsPP[:10], doc_clean[:10], tweetsClean[:10]):
    print("tw: [%s]" % t)
    print("pp: [%s]" % p)
    print("cl: [%s]" % dc)
    print("tc: [%s]" % tc)
    print()
#print(tweets)

tw: [RT @CharlesHDean: A great resource if you're curious about dental implants. https://t.co/WRHJKNHGfR]
pp: [rt mentionname a great resource if you re curious about dental implants urlurl ]
cl: [['mentionnam', 'great', 'resourc', 'curious', 'urlurl']]
tc: [mentionnam great resourc curious urlurl]

tw: [Melbourne dental implant failure and how to avoid it https://t.co/620oDESzyg https://t.co/3qKZO6EVVn]
pp: [melbourne dental implant failure and how to avoid it urlurl ]
cl: [['melbourn', 'failur', 'avoid', 'urlurl']]
tc: [melbourn failur avoid urlurl]

tw: [RT @GuyKawasaki: Ceramic dental implants get crushed by a 20-ton press https://t.co/YVyROKk9bu https://t.co/m1pK8BJM4o]
pp: [rt mentionname ceramic dental implants get crushed by a numbers ton press urlurl ]
cl: [['mentionnam', 'ceram', 'crush', 'number', 'press', 'urlurl']]
tc: [mentionnam ceram crush number press urlurl]

tw: [RT @ukbestdentist: Full Set Dental Implants in The Blythe #Complete #Teeth #Implants #The #Blythe https:/

## Testing scikit

In [29]:
import string
import numpy as np

from joblib import Parallel, delayed
import multiprocessing

from nltk.stem import PorterStemmer
from nltk import word_tokenize


from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

import sklearn.cross_validation
from sklearn.model_selection import cross_val_score

In [30]:
from sklearn.cross_validation import train_test_split
 
def train(classifier, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
    classifier.fit(X_train, y_train)
    print("Accuracy: %s" % classifier.score(X_test, y_test))
    return classifier

In [31]:
# MultinomialNB
MNBtrials = []
cont = 0
contAll = 0
allTrials = []

for i in range(0, 11, 1):
    alpha = i/10
    for j in range(5, 11, 1):
        max_df = j/10
        for k in range(1, 11, 1):
            min_df = k
            trial = Pipeline([
                ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
                ('classifier', MultinomialNB(alpha=alpha))
                ])
            desc = 'MNB [alpha: ' + str(alpha) + ' max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
            MNBtrials.append((cont,desc,trial))
            cont+=1
            allTrials.append((contAll,desc,trial))
            contAll+=1

for t in MNBtrials[:10]:
    print(t[0],t[1])
print(len(MNBtrials))
print('--')
print(MNBtrials[2][1])
print('--')
print(len(allTrials))
print(allTrials[0])
print(allTrials[659])

0 MNB [alpha: 0.0 max_df:0.5 min_df: 1]
1 MNB [alpha: 0.0 max_df:0.5 min_df: 2]
2 MNB [alpha: 0.0 max_df:0.5 min_df: 3]
3 MNB [alpha: 0.0 max_df:0.5 min_df: 4]
4 MNB [alpha: 0.0 max_df:0.5 min_df: 5]
5 MNB [alpha: 0.0 max_df:0.5 min_df: 6]
6 MNB [alpha: 0.0 max_df:0.5 min_df: 7]
7 MNB [alpha: 0.0 max_df:0.5 min_df: 8]
8 MNB [alpha: 0.0 max_df:0.5 min_df: 9]
9 MNB [alpha: 0.0 max_df:0.5 min_df: 10]
660
--
MNB [alpha: 0.0 max_df:0.5 min_df: 3]
--
660
(0, 'MNB [alpha: 0.0 max_df:0.5 min_df: 1]', Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...      vocabulary=None)), ('classifier', MultinomialNB(alpha=0.0, class_prior=None, fit_prior=True))]))
(659, 'MNB [alpha: 1.0 max_df:1.0 min_df: 10]', Pipeline(memory=None,

In [32]:
# Random Forest
RFtrials = []
cont = 0

for val in range(1, 41):
    for j in range(5, 11, 1):
        max_df = j/10
        for k in range(1, 11, 1):
            min_df = k
            trial = Pipeline([
                ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
                ('classifier', RandomForestClassifier(n_estimators=val,random_state=10)),
                ])
            desc = 'RF [val: ' + str(alpha) + ' max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
            RFtrials.append((cont,desc,trial))
            cont+=1
            allTrials.append((contAll,desc,trial))
            contAll+=1            

for t in RFtrials[:12]:
    print(t[0],t[1])
print(len(RFtrials))
print('--')
print(RFtrials[2][1])


#for val in range(1,41):
#    trial = Pipeline([
#        ('vectorizer', TfidfVectorizer(min_df=5, max_df=0.9)),
#        ('classifier', RandomForestClassifier(n_estimators=val)),
#    ])
#    trials.append(trial)
print('--')
print(len(allTrials))
print(allTrials[0])
print(allTrials[659])

0 RF [val: 1.0 max_df:0.5 min_df: 1]
1 RF [val: 1.0 max_df:0.5 min_df: 2]
2 RF [val: 1.0 max_df:0.5 min_df: 3]
3 RF [val: 1.0 max_df:0.5 min_df: 4]
4 RF [val: 1.0 max_df:0.5 min_df: 5]
5 RF [val: 1.0 max_df:0.5 min_df: 6]
6 RF [val: 1.0 max_df:0.5 min_df: 7]
7 RF [val: 1.0 max_df:0.5 min_df: 8]
8 RF [val: 1.0 max_df:0.5 min_df: 9]
9 RF [val: 1.0 max_df:0.5 min_df: 10]
10 RF [val: 1.0 max_df:0.6 min_df: 1]
11 RF [val: 1.0 max_df:0.6 min_df: 2]
2400
--
RF [val: 1.0 max_df:0.5 min_df: 3]
--
3060
(0, 'MNB [alpha: 0.0 max_df:0.5 min_df: 1]', Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...      vocabulary=None)), ('classifier', MultinomialNB(alpha=0.0, class_prior=None, fit_prior=True))]))
(659, 'MNB [alpha: 1.0 m

In [33]:
# SGDClassifier
SGDtrials = []
cont = 0
lr = 1
for i in range(1,10):
    lr = lr / 10

    for j in range(5, 11, 1):
        max_df = j/10
        for k in range(1, 11, 1):
            min_df = k

            # SGD
            trial = Pipeline([
                ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
                ('classifier', SGDClassifier(random_state=10)),
                ])
            desc = 'SGD [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ' learning_rate : ' +str(lr) + ']'
            SGDtrials.append((cont,desc,trial))
            cont+=1
            allTrials.append((contAll,desc,trial))
            contAll+=1            
            
for j in range(5, 11, 1):
    max_df = j/10
    for k in range(1, 11, 1):
        min_df = k

        # SGD
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', SGDClassifier(random_state=10)),
            ])
        desc = 'SGD [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        SGDtrials.append((cont,desc,trial))
        cont+=1
        allTrials.append((contAll,desc,trial))
        contAll+=1            
            
            
            
for t in SGDtrials:
    print(t[0],t[1])
print(len(SGDtrials))
print('--')
print(SGDtrials[2][1])


0 SGD [max_df:0.5 min_df: 1 learning_rate : 0.1]
1 SGD [max_df:0.5 min_df: 2 learning_rate : 0.1]
2 SGD [max_df:0.5 min_df: 3 learning_rate : 0.1]
3 SGD [max_df:0.5 min_df: 4 learning_rate : 0.1]
4 SGD [max_df:0.5 min_df: 5 learning_rate : 0.1]
5 SGD [max_df:0.5 min_df: 6 learning_rate : 0.1]
6 SGD [max_df:0.5 min_df: 7 learning_rate : 0.1]
7 SGD [max_df:0.5 min_df: 8 learning_rate : 0.1]
8 SGD [max_df:0.5 min_df: 9 learning_rate : 0.1]
9 SGD [max_df:0.5 min_df: 10 learning_rate : 0.1]
10 SGD [max_df:0.6 min_df: 1 learning_rate : 0.1]
11 SGD [max_df:0.6 min_df: 2 learning_rate : 0.1]
12 SGD [max_df:0.6 min_df: 3 learning_rate : 0.1]
13 SGD [max_df:0.6 min_df: 4 learning_rate : 0.1]
14 SGD [max_df:0.6 min_df: 5 learning_rate : 0.1]
15 SGD [max_df:0.6 min_df: 6 learning_rate : 0.1]
16 SGD [max_df:0.6 min_df: 7 learning_rate : 0.1]
17 SGD [max_df:0.6 min_df: 8 learning_rate : 0.1]
18 SGD [max_df:0.6 min_df: 9 learning_rate : 0.1]
19 SGD [max_df:0.6 min_df: 10 learning_rate : 0.1]
20 SGD [

In [34]:
# SGDClassifier
SGDtrials = []
LinearSVCtrials = []
SVCtrials = []
cont = 0

for j in range(5, 11, 1):
    max_df = j/10
    for k in range(1, 11, 1):
        min_df = k
        
        # SGD
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', SGDClassifier(random_state=10)),
            ])
        desc = 'SGD [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        SGDtrials.append((cont,desc,trial))
        allTrials.append((contAll,desc,trial))
        contAll+=1            
        
        
        # Linear SVC
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', LinearSVC()),
            ])
        desc = 'LinearSVC [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        LinearSVCtrials.append((cont,desc,trial))
        allTrials.append((contAll,desc,trial))
        contAll+=1            
        
        # SVC
        trial = Pipeline([
            ('vectorizer', TfidfVectorizer(min_df=min_df, max_df=max_df)),
            ('classifier', SVC()),
            ])
        desc = 'SVC [max_df:' + str(max_df) + ' min_df: ' + str(min_df) + ']'
        SVCtrials.append((cont,desc,trial))
        allTrials.append((contAll,desc,trial))
        contAll+=1            
        
        
        cont+=1


for t in SGDtrials[:10]:
    print(t[0],t[1])
print(len(SGDtrials))
print('--')
print(SGDtrials[2][1])

print('\n\n')

for t in LinearSVCtrials[:10]:
    print(t[0],t[1])
print(len(LinearSVCtrials))
print('--')
print(LinearSVCtrials[2][1])

print('\n\n')


for t in SVCtrials[:10]:
    print(t[0],t[1])
print(len(SVCtrials))
print('--')
print(SVCtrials[2][1])

0 SGD [max_df:0.5 min_df: 1]
1 SGD [max_df:0.5 min_df: 2]
2 SGD [max_df:0.5 min_df: 3]
3 SGD [max_df:0.5 min_df: 4]
4 SGD [max_df:0.5 min_df: 5]
5 SGD [max_df:0.5 min_df: 6]
6 SGD [max_df:0.5 min_df: 7]
7 SGD [max_df:0.5 min_df: 8]
8 SGD [max_df:0.5 min_df: 9]
9 SGD [max_df:0.5 min_df: 10]
60
--
SGD [max_df:0.5 min_df: 3]



0 LinearSVC [max_df:0.5 min_df: 1]
1 LinearSVC [max_df:0.5 min_df: 2]
2 LinearSVC [max_df:0.5 min_df: 3]
3 LinearSVC [max_df:0.5 min_df: 4]
4 LinearSVC [max_df:0.5 min_df: 5]
5 LinearSVC [max_df:0.5 min_df: 6]
6 LinearSVC [max_df:0.5 min_df: 7]
7 LinearSVC [max_df:0.5 min_df: 8]
8 LinearSVC [max_df:0.5 min_df: 9]
9 LinearSVC [max_df:0.5 min_df: 10]
60
--
LinearSVC [max_df:0.5 min_df: 3]



0 SVC [max_df:0.5 min_df: 1]
1 SVC [max_df:0.5 min_df: 2]
2 SVC [max_df:0.5 min_df: 3]
3 SVC [max_df:0.5 min_df: 4]
4 SVC [max_df:0.5 min_df: 5]
5 SVC [max_df:0.5 min_df: 6]
6 SVC [max_df:0.5 min_df: 7]
7 SVC [max_df:0.5 min_df: 8]
8 SVC [max_df:0.5 min_df: 9]
9 SVC [max_df:0.5 m

In [35]:
print(len(SGDtrials))
print(len(LinearSVCtrials))
print(len(SVCtrials))
print(len(MNBtrials))
print(len(RFtrials))

allTrials = SGDtrials + LinearSVCtrials + SVCtrials + MNBtrials + RFtrials
print(len(allTrials))

60
60
60
660
2400
3240


In [36]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn import model_selection

from sklearn.metrics import  make_scorer
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=42)
model=allTrials[928][2]

#results = model_selection.cross_val_score(estimator=model,
#                                          X=features,
#                                          y=labels,
#                                          cv=kfold,
#                                          scoring='f1_micro'

In [37]:
#%%writefile functions/classificacao.py
def classifierScore(classifier, X_train, y_train, X, y, metrica, idx):
    
    scoring = {'accuracy' : make_scorer(accuracy_score), 
               'precision' : make_scorer(precision_score),
               'recall' : make_scorer(recall_score), 
               'f1' : make_scorer(f1_score)}
    scoring = ['accuracy', 'precision']
    scoring = 'f1_micro'

    kfold = model_selection.KFold(n_splits=10, random_state=42)
    
    #print(classifier[1])
    classifier[2].fit(X_train, y_train)
    scores = model_selection.cross_val_score(classifier[2], X, y, cv=kfold, scoring=metrica)
    return (idx, scores)

def bestClassifier(X, y, classifiers, metrica):
    target_names = list(set(y))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=50)
    maxScore = 0
    ic = 0
    
    
    #for idx, val in enumerate(allTrials):
        
    num_cores = multiprocessing.cpu_count()
    resultados = Parallel(n_jobs=num_cores)(delayed(classifierScore)(classifier, X_train, y_train, X, y, metrica, idx) 
                                            for idx, classifier in enumerate(classifiers))
                                            #for classifier in classifiers)
    #print(resultados)

    for result in resultados:
        classifier = classifiers[result[0]]

        scores = result[1]
        #print(scores)

        score = scores.mean()
        std = scores.std() * 2

        if score > maxScore:
            maxScore = score
            maxStd = std
            maxIc = ic
            bClassifier = classifier
        ic += 1

    print(bClassifier[1], '[', bClassifier[0], ']')
    print("max: ", maxScore)
    print("%s: %0.2f (+/- %0.2f)" % (metrica, maxScore, maxStd))
    
    print('----')
    
    y_true = []
    y_pred = []

    y_true = y_test
    classifiers[bClassifier[0]][2].fit(X_train, y_train)
    y_pred = classifiers[bClassifier[0]][2].predict(X_test)
    report2 = sklearn.metrics.classification_report(y_true, y_pred)
    print(report2)
    
    
    return bClassifier

# Melhor Classificador Dental Implants (tweets pré-processados):

In [38]:
print(len(allTrials))

3240


In [52]:
allTrials[111]

(51, 'LinearSVC [max_df:1.0 min_df: 2]', Pipeline(memory=None,
      steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=2,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ax_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0))]))

In [140]:
a = bestClassifier(tweetsPP, topics, allTrials, 'f1_weighted')

LinearSVC [max_df:1.0 min_df: 2] [ 51 ]
max:  0.5243856210747079
f1_weighted: 0.52 (+/- 0.08)
----
             precision    recall  f1-score   support

   clinical       0.40      0.32      0.36        60
information       0.43      0.48      0.45        25
   personal       0.44      0.31      0.36        39
    product       0.21      0.26      0.23        35
       sale       0.30      0.14      0.19        21
    service       0.51      0.62      0.56       120

avg / total       0.42      0.43      0.42       300



In [141]:
b = bestClassifier(tweets, topics, allTrials, 'f1_weighted')

LinearSVC [max_df:0.9 min_df: 1] [ 40 ]
max:  0.520194393805159
f1_weighted: 0.52 (+/- 0.11)
----
             precision    recall  f1-score   support

   clinical       0.40      0.37      0.38        60
information       0.52      0.48      0.50        25
   personal       0.50      0.31      0.38        39
    product       0.21      0.14      0.17        35
       sale       0.29      0.10      0.14        21
    service       0.52      0.72      0.61       120

avg / total       0.44      0.47      0.44       300



# Melhor Classificador Dental Implants (tweets pré-processados):

In [142]:
c = bestClassifier(tweetsPP, topics, allTrials, 'accuracy')

MNB [alpha: 0.3 max_df:0.9 min_df: 3] [ 222 ]
max:  0.5385959595959596
accuracy: 0.54 (+/- 0.10)
----
             precision    recall  f1-score   support

   clinical       0.39      0.18      0.25        60
information       0.55      0.44      0.49        25
   personal       0.41      0.28      0.33        39
    product       0.33      0.14      0.20        35
       sale       0.57      0.19      0.29        21
    service       0.47      0.79      0.59       120

avg / total       0.44      0.46      0.41       300



In [143]:
d = bestClassifier(tweets, topics, allTrials, 'accuracy')

LinearSVC [max_df:0.9 min_df: 1] [ 40 ]
max:  0.5456060606060606
accuracy: 0.55 (+/- 0.11)
----
             precision    recall  f1-score   support

   clinical       0.40      0.37      0.38        60
information       0.52      0.48      0.50        25
   personal       0.50      0.31      0.38        39
    product       0.21      0.14      0.17        35
       sale       0.29      0.10      0.14        21
    service       0.52      0.72      0.61       120

avg / total       0.44      0.47      0.44       300



## Matriz de Confusão

In [122]:
yga = [ 0,1,2,3,4,5,6,7,8,9]
print(yga[:3])
print(yga[3:])

[0, 1, 2]
[3, 4, 5, 6, 7, 8, 9]


In [123]:
len(tweetsPP[:697])

697

In [144]:
a

(51, 'LinearSVC [max_df:1.0 min_df: 2]', Pipeline(memory=None,
      steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=2,
         ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ax_iter=1000,
      multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
      verbose=0))]))

In [57]:
bestTrial = allTrials[55]
bestTrial[2].fit(tweetsPP[:697], topics[:697])


Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=6,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ty='l2', power_t=0.5, random_state=10, shuffle=True,
       tol=None, verbose=0, warm_start=False))])

In [58]:
y_pred = bestTrial[2].predict(tweetsPP[697:])
for a in Counter(y_pred).most_common():
    print(a)

('service', 103)
('personal', 72)
('product', 58)
('information', 37)
('clinical', 16)
('sale', 14)


In [85]:
y_true = topics[697:]
for a in Counter(y_true).most_common():
    print(a)

('service', 83)
('service clinical', 50)
('personal', 44)
('clinical', 39)
('product', 31)
('information', 31)
('sale', 22)


### Opções:

- remover de vez as categorias
- buscar no total de tweets, mensagens que sejam classificadas nessas categorias




## Executando classificador em todos os tweets

In [145]:
# all tweets
cmd = ("select count(*) "
       "from amostratweet at "
       "inner join tweet t on at.codtweet = t.codtweet "
       "where codamostra = 132")
query(cmd)

[(222603,)]

In [53]:
cmd = ("select t.codtweet, t.text, 0 "
       "from amostratweet at "
       "inner join tweet t on at.codtweet = t.codtweet "
       "where codamostra = 132")
clTweets = query(cmd)

In [54]:
clTweets[:10]

[(77500292,
  'Dental Implants Cost: 5 Factors that Affecting Implant Cost 0\n#dental #implants #cost\nhttps://t.co/zgsvqj7k71',
  0),
 (77500308,
  'Dental implants bone grafting sucks! So greatful for CCF Dentistry they are world class! https://t.co/Nw1h9gxcas',
  0),
 (77500338,
  'Cheap Dental Implants: 5 Tips to Find the Cheap Implants 0\n#Cheap #dentalimplants #implants\nhttps://t.co/WORzqig9hK',
  0),
 (77500339,
  'Cheap Dental Implants: 5 Tips to Find the Cheap Implants 0\n#Cheap #dentalimplants #implants\nhttps://t.co/5LS2ku0sva',
  0),
 (77500377,
  '@RevDavidColes Nurofen Plus. I just had 4 dental implants at once, so I know it works!',
  0),
 (77500385,
  'Don’t miss - Dental implants: survival rates of turned and anodised-surface dental implants https://t.co/fqV5BsCHrD',
  0),
 (77559781,
  'Check out Dental Implant Broken Screw Remover &amp; Repair Kit  EMS Biomet 3I #totalimplantcom https://t.co/RDv6tEeiAL via @eBay',
  0),
 (77559785,
  'Check out Dental Implant Broken

In [55]:
allTweets = []
for t in clTweets:
    allTweets.append(t[1])
allTweetsPP = pre_proc(allTweets)

In [56]:
doc_clean = [clean(doc).split() for doc in allTweetsPP] 
allTweetsClean = []
for t in doc_clean:
    allTweetsClean.append(" ".join([i for i in t]))

In [57]:
print(len(clTweets))
print(len(allTweets))
print(len(allTweetsPP))
print(len(doc_clean))
print(len(allTweetsClean))


for cl, t, p, dc, tc in zip(clTweets[:10], allTweets[:10], allTweetsPP[:10], doc_clean[:100], allTweetsClean[:100]):
    print("[%s]" % cl[0])
    print("[%s]" % cl[1])
    print("[%s]" % t)
    print("[%s]" % p)
    print("[%s]" % dc)
    print("[%s]" % tc)
    print('---\n')
#print(tweets)


222603
222603
222603
222603
222603
[77500292]
[Dental Implants Cost: 5 Factors that Affecting Implant Cost 0
#dental #implants #cost
https://t.co/zgsvqj7k71]
[Dental Implants Cost: 5 Factors that Affecting Implant Cost 0
#dental #implants #cost
https://t.co/zgsvqj7k71]
[dental implants cost numbers factors that affecting implant cost numbers dental implants cost urlurl ]
[['cost', 'number', 'factor', 'affect', 'cost', 'number', 'cost', 'urlurl']]
[cost number factor affect cost number cost urlurl]
---

[77500308]
[Dental implants bone grafting sucks! So greatful for CCF Dentistry they are world class! https://t.co/Nw1h9gxcas]
[Dental implants bone grafting sucks! So greatful for CCF Dentistry they are world class! https://t.co/Nw1h9gxcas]
[dental implants bone grafting sucks so greatful for ccf dentistry they are world class urlurl ]
[['bone', 'graft', 'suck', 'great', 'dentistri', 'world', 'class', 'urlurl']]
[bone graft suck great dentistri world class urlurl]
---

[77500338]
[Cheap 

In [151]:
tweetsClean[:10]

['know overal success rate number percent questionmark beat odd read urlurl',
 'singl tooth procedur tooth surgeri urlurl',
 'accord american academi dentistri possess high success rate number urlurl',
 'abut system garner moneymark number number number persist market research urlurl',
 'come tooth replac option best longev practic thes urlurl',
 'mentionnam done procedur questionmark learn advantag booost urlurl',
 'abut analog urlurl',
 'free inform booklet therapi creat common question hear regard urlurl',
 'number wine houston',
 'clinic bloom energi fuel cell']

In [58]:
bestTrial = allTrials[111]
bestTrial[2].fit(tweetsClean, topics)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=Tr...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [59]:
y_pred = bestTrial[2].predict(allTweetsClean)
for a in Counter(y_pred).most_common():
    print(a)

('service', 113146)
('clinical', 40214)
('personal', 23634)
('product', 18049)
('information', 17227)
('sale', 10333)


In [154]:
y_pred = bestTrial[2].predict(allTweetsClean)
for a in Counter(y_pred).most_common():
    print(a)

('service', 113146)
('clinical', 40214)
('personal', 23634)
('product', 18049)
('information', 17227)
('sale', 10333)


In [60]:
for a in Counter(y_pred).most_common():
    print(a[0],';',a[1])

service ; 113146
clinical ; 40214
personal ; 23634
product ; 18049
information ; 17227
sale ; 10333


In [61]:
len(y_pred)

222603

In [62]:
len(y_pred)

222603

In [68]:

for cl, allt, yp in zip(clTweets, allTweetsClean, y_pred):
    #print("[%s]" % cl[0])
    #print("[%s]" % cl[1])
    #print("[%s]" % allt)
    #print("[%s]" % yp)
    
    codcl = 39
    if yp == 'information': codcl = 223
    if yp == 'product': codcl = 224
    if yp == 'service': codcl = 226
    if yp == 'personal': codcl = 227
    if yp == 'sale': codcl = 228
    if yp == 'clinical': codcl = 229
    if yp == 'service clinical' : codcl = 230
    
    #if codcl != 39: continue
    #print("[%s]" % dc)
    #print("[%s]" % tc)
    cmd = ("INSERT INTO resultadoanaliseprojetotweet "
           "(codresultadoanaliseprojeto, codclassificacao, codtweet) VALUES " 
           "(12, " + str(codcl) + " , " + str(cl[0]) + ")");
    #print(cmd)
    execSQL(cmd)
    
    #print('---\n')
#print(tweets)


In [72]:
cmd = ("SELECT "
       "cl.descricao, "
       "to_char(datatweet, 'YYYY-MM') as data, "
       "count(r.codtweet) "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "INNER JOIN tweet t ON r.codtweet = t.codtweet "
       "where codresultadoanaliseprojeto = 12 "
       "GROUP BY 1, 2 "
       "ORDER BY 1, 2"
      )
res = query(cmd)
for r in res:
    print(r[0],";",r[1],";",r[2])

clinical ; 2016-07 ; 1336
clinical ; 2016-08 ; 1338
clinical ; 2016-09 ; 1287
clinical ; 2016-10 ; 2611
clinical ; 2016-11 ; 1360
clinical ; 2016-12 ; 1681
clinical ; 2017-01 ; 4132
clinical ; 2017-02 ; 2458
clinical ; 2017-03 ; 1926
clinical ; 2017-04 ; 1158
clinical ; 2017-05 ; 1368
clinical ; 2017-06 ; 1644
clinical ; 2017-07 ; 1734
clinical ; 2017-08 ; 1543
clinical ; 2017-09 ; 1760
clinical ; 2017-10 ; 1502
clinical ; 2017-11 ; 1305
clinical ; 2017-12 ; 1194
clinical ; 2018-01 ; 1401
clinical ; 2018-02 ; 1276
clinical ; 2018-03 ; 1174
clinical ; 2018-04 ; 773
clinical ; 2018-05 ; 152
clinical ; 2018-07 ; 175
clinical ; 2018-08 ; 412
clinical ; 2018-09 ; 823
clinical ; 2018-10 ; 1084
clinical ; 2018-11 ; 872
clinical ; 2018-12 ; 735
information ; 2016-07 ; 175
information ; 2016-08 ; 460
information ; 2016-09 ; 647
information ; 2016-10 ; 344
information ; 2016-11 ; 775
information ; 2016-12 ; 837
information ; 2017-01 ; 1280
information ; 2017-02 ; 517
information ; 2017-03 ; 812


In [74]:
ks = list(range(10, 100, 10)) + list(range(100, 300, 20)) + list(range(300, 500, 50)) + [500, 600, 700]
len(ks)

26